In [1]:
import torch

from dataclasses import dataclass
from functools import partial
from typing import Any, Dict, List, Union

from datasets import load_dataset, DatasetDict, Audio
from evaluate import load
from transformers import pipeline
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset
from transformers.models.whisper.english_normalizer import BasicTextNormalizer, EnglishTextNormalizer
from transformers.models.whisper.tokenization_whisper import TO_LANGUAGE_CODE
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

## Hyper Parameters mainly changed

In [2]:
MODEL_NAME = "openai/whisper-small"
STEPS = 500

## Boiler plate code

In [3]:
poly = load_dataset(
    "PolyAI/minds14", "en-US", split="train"
)

Found cached dataset minds14 (/home/.cache/huggingface/datasets/PolyAI___minds14/en-US/1.0.0/65c7e0f3be79e18a6ffaf879a083daf706312d421ac90d25718459cbf3c42696)


In [4]:
poly

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 563
})

In [5]:
451/563

0.8010657193605684

In [6]:
poly = DatasetDict()

poly["train"] = load_dataset(
    "PolyAI/minds14", "en-US", split="train[0%:80%]"
)
poly["test"] = load_dataset(
    "PolyAI/minds14", "en-US", split="train[80%:100%]"
)

print(poly)

Found cached dataset minds14 (/home/.cache/huggingface/datasets/PolyAI___minds14/en-US/1.0.0/65c7e0f3be79e18a6ffaf879a083daf706312d421ac90d25718459cbf3c42696)
Found cached dataset minds14 (/home/.cache/huggingface/datasets/PolyAI___minds14/en-US/1.0.0/65c7e0f3be79e18a6ffaf879a083daf706312d421ac90d25718459cbf3c42696)


DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 450
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 113
    })
})


In [7]:
poly = poly.select_columns(["audio", "transcription"])

In [8]:
processor = WhisperProcessor.from_pretrained(
    MODEL_NAME, language="english", task="transcribe"
)

if torch.cuda.is_available():
    device = "cuda:0"
    torch_dtype = torch.float16
else:
    device = "cpu"
    torch_dtype = torch.float32

pipe = pipeline(
    "automatic-speech-recognition",
    model=MODEL_NAME,
    torch_dtype=torch_dtype,
    device=device,
)

In [9]:
poly["train"].features

{'audio': Audio(sampling_rate=8000, mono=True, decode=True, id=None),
 'transcription': Value(dtype='string', id=None)}

In [10]:
sampling_rate = processor.feature_extractor.sampling_rate
sampling_rate

16000

In [11]:
poly = poly.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [12]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["transcription"],
    )

    # compute input length of audio sample in seconds
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

In [13]:
%%time
poly = poly.map(
    prepare_dataset, remove_columns=poly.column_names["train"]
)

Loading cached processed dataset at /home/.cache/huggingface/datasets/PolyAI___minds14/en-US/1.0.0/65c7e0f3be79e18a6ffaf879a083daf706312d421ac90d25718459cbf3c42696/cache-c54bab2f9fa7b7e8.arrow


Map:   0%|          | 0/113 [00:00<?, ? examples/s]

CPU times: user 17.2 s, sys: 576 ms, total: 17.8 s
Wall time: 10.6 s


In [14]:
max_input_length = 30.0


def is_audio_in_length_range(length):
    return length < max_input_length

In [15]:
poly["train"] = poly["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)
poly["train"]

Loading cached processed dataset at /home/.cache/huggingface/datasets/PolyAI___minds14/en-US/1.0.0/65c7e0f3be79e18a6ffaf879a083daf706312d421ac90d25718459cbf3c42696/cache-f9a69689bb639982.arrow


Dataset({
    features: ['input_features', 'labels', 'input_length'],
    num_rows: 445
})

In [16]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [17]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [18]:
def compute_metrics(pred):
    normalizer = BasicTextNormalizer()
    wer_metric = load("wer")
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = 100 * wer_metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = 100 * wer_metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [19]:
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)

In [20]:
# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language="english", task="transcribe", use_cache=True
)

In [21]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./hfa-poly_english_small",  # name on the HF Hub
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=STEPS,  # increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [22]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=poly["train"],
    eval_dataset=poly["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

/home/hf_audio/./hfa-poly_english_small is already a clone of https://huggingface.co/kurianbenoy/hfa-poly_english_small. Make sure you pull the latest changes with `repo.git_pull()`.


In [23]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


TypeError: EnglishTextNormalizer.__init__() missing 1 required positional argument: 'english_spelling_mapping'